In [0]:
# My repo with data
!git clone https://github.com/Tixonmavrin/covid-19-solution
!unzip covid-19-solution/data/data_with_features/_data_with_features.csv.zip
!unzip covid-19-solution/data/data_with_features/_data_with_features_all.csv.zip

In [0]:
import pandas as pd
import numpy as np
import warnings
import copy
import lightgbm as lgb
from sklearn import metrics
from tqdm import tqdm
import random
pd.set_option('display.max_columns', 100)
warnings.filterwarnings("ignore")

In [0]:
#data4 = pd.read_csv('_data_with_features.csv')
data4 = pd.read_csv('_data_with_features_all.csv')

In [0]:
data4['Country/Region'] = data4['Country/Region'].fillna('nan').astype('str')
data4['Province/State'] = data4['Province/State'].fillna('nan').astype('str')
data4['Date'] = pd.to_datetime(data4['Date'])
data4['day'] = data4['Date'].apply(lambda x: x.dayofyear).astype(np.int16)

In [0]:
#data4 = data4[data4['Country/Region'] != 'US']
data4['place'] = data4['Country/Region'].fillna('') + '/' + data4['Province/State'].fillna('')
places = data4['place'].unique()
# Cummax
data4.loc[data4['Country/Region'] != 'Russia','Confirmed'] = data4.loc[data4['Country/Region'] != 'Russia'].groupby("place")["Confirmed"].cummax()
data4.loc[data4['Country/Region'] != 'Russia','Deaths'] = data4.loc[data4['Country/Region'] != 'Russia'].groupby("place")["Deaths"].cummax()

In [38]:
data4n = []
for place in tqdm(data4['place'].unique()):
  if ('Russia' in place) and (',' not in place) and (place.count('Russia') == 1):
    data4n.append(data4[data4['place'] == place])
  else:
    data4c = data4[data4['place'] == place]
    if data4c['Confirmed'].nunique() > 60:
      data4n.append(data4c)
data4 = pd.concat(data4n)

100%|██████████| 2178/2178 [00:33<00:00, 65.68it/s]


In [39]:
places = data4['place'].unique()
data4['Confirmed per day'] = 0
temp_list = np.zeros(len(data4))
for place in tqdm(places):
    temp = data4['Confirmed'][data4['place']==place].values
    temp[1:] = temp[1:] - temp[:-1]
    data4['Confirmed per day'][data4['place']==place] = temp

100%|██████████| 313/313 [00:02<00:00, 131.64it/s]


In [0]:
def aggregate(data_agg, column, left_right, function, name=''):
    data_agg_new = data_agg.copy()
    column_name = '{}_{}_{}_{}'.format(column, name, left_right[0], left_right[1])
    data_agg_new[column_name] = 0
    tmp = data_agg_new[column].rolling(left_right[1]-left_right[0]+1).agg(function)
    data_agg_new[column_name][left_right[0]:] = tmp[:-left_right[0]]
    return data_agg_new

def make_features(data_agg):
    data_agg = aggregate(data_agg, 'Confirmed per day', [1,1], 'mean', 'mean')
    data_agg = aggregate(data_agg, 'Confirmed per day', [1,7], 'mean','mean')
    data_agg = aggregate(data_agg, 'Confirmed per day', [8,14], 'mean','mean')
    data_agg = aggregate(data_agg, 'Confirmed per day', [15,21], 'mean','mean')

    data_agg = aggregate(data_agg, 'Confirmed per day', [1,7], 'max','max')
    data_agg = aggregate(data_agg, 'Confirmed per day', [1,7], 'min', 'min')
    data_agg = aggregate(data_agg, 'Confirmed per day', [1,7], 'median', 'median')
    
    for thresh in [1, 10, 100]:
        days_under_thresh = (data_agg['Confirmed']<thresh).sum()
        tmp = data_agg['day'].values - days_under_thresh
        tmp[tmp<=0] = 0
        data_agg['days_from_{}'.format(thresh)] = tmp

    for lag in range(1, 14):
        data_agg[f"lag_{lag}_cc"] = data_agg.groupby("place")["Confirmed"].shift(lag)

    data_agg["perc_1_cc"] = data_agg[f"lag_1_cc"] / data_agg.population
    
    data_agg["diff_1_cc"] = data_agg[f"lag_1_cc"] - data_agg[f"lag_2_cc"]
    data_agg["diff_2_cc"] = data_agg[f"lag_2_cc"] - data_agg[f"lag_3_cc"]
    data_agg["diff_3_cc"] = data_agg[f"lag_3_cc"] - data_agg[f"lag_4_cc"]
    
    data_agg["diff_123_cc"] = (data_agg[f"lag_1_cc"] - data_agg[f"lag_4_cc"]) / 3

    data_agg["diff_change_1_cc"] = data_agg.diff_1_cc / data_agg.diff_2_cc
    data_agg["diff_change_2_cc"] = data_agg.diff_2_cc / data_agg.diff_3_cc

    data_agg["diff_change_12_cc"] = (data_agg.diff_change_1_cc + data_agg.diff_change_2_cc) / 2

    data_agg["change_1_cc"] = data_agg[f"lag_1_cc"] / data_agg[f"lag_2_cc"]
    data_agg["change_2_cc"] = data_agg[f"lag_2_cc"] / data_agg[f"lag_3_cc"]
    data_agg["change_3_cc"] = data_agg[f"lag_3_cc"] / data_agg[f"lag_4_cc"]

    data_agg["change_1_3_cc"] = data_agg[f"lag_1_cc"] / data_agg[f"lag_4_cc"]
    data_agg["change_1_7_cc"] = data_agg[f"lag_1_cc"] / data_agg[f"lag_8_cc"]

    data_agg.reset_index(drop=True, inplace=True)
    data_agg["day_from_max"] = 0
    data_agg["max_value"] = 0
    data_agg["delta_with_max"] = 0
    vmax = 0.0
    imax = 0
    for i in range(1, data_agg.shape[0]):
      if data_agg.loc[i-1, 'Confirmed per day'] > vmax:
        vmax = data_agg.loc[i-1, 'Confirmed per day']
        imax = i-1
      data_agg.loc[i, 'delta_with_max'] = data_agg.loc[i,'Confirmed per day'] - vmax
      data_agg.loc[i, 'max_value'] = vmax
      data_agg.loc[i, 'day_from_max'] = i - imax
    
    data_agg["day_from_max_1"] = data_agg.groupby("place")["day_from_max"].shift(1)
    data_agg["max_value_1"] = data_agg.groupby("place")["max_value"].shift(1)
    data_agg["delta_with_max_1"] = data_agg.groupby("place")["delta_with_max"].shift(1)

    data_agg["apl-driving_8"] = data_agg.groupby("place")["apl-driving"].shift(8)
    data_agg["apl-transit_8"] = data_agg.groupby("place")["apl-transit"].shift(8)
    data_agg["apl-walking_8"] = data_agg.groupby("place")["apl-walking"].shift(8)
    data_agg["grocery-and-pharmacy_8"] = data_agg.groupby("place")["grocery-and-pharmacy"].shift(8)
    data_agg["parks_8"] = data_agg.groupby("place")["parks"].shift(8)
    data_agg["residential_8"] = data_agg.groupby("place")["residential"].shift(8)
    data_agg["retail-and-recreation_8"] = data_agg.groupby("place")["retail-and-recreation"].shift(8)
    data_agg["transit-stations_8"] = data_agg.groupby("place")["transit-stations"].shift(8)
    data_agg["workplaces_8"] = data_agg.groupby("place")["workplaces"].shift(8)
    data_agg["isolation_8"] = data_agg.groupby("place")["isolation"].shift(8)

    return data_agg

In [41]:
data5 = []
for place in tqdm(places[:]):
    temp = data4[data4['place']==place].reset_index(drop=True)
    temp = make_features(temp)
    data5.append(temp)
data5 = pd.concat(data5).reset_index(drop=True)

100%|██████████| 313/313 [01:04<00:00,  4.89it/s]


In [42]:
MIN_VALID = '2020-04-20'
MAX_VALID = '2020-04-26'
MIN_TEST = '2020-04-27'
MAX_TEST = '2020-05-03'

data5['Date'] = pd.to_datetime(data5['Date'])
data5.columns = ['Country/Region', 'Province/State', 'Date', 'Confirmed', 'Deaths'] + list(data5.columns[5:])
traint = data5[data5['Date'] < pd.to_datetime(MIN_TEST)]
test = data5[(data5['Date'] >= pd.to_datetime(MIN_TEST)) & (data5['Date'] <= pd.to_datetime(MAX_TEST))]
trainv = data5[data5['Date'] < pd.to_datetime(MIN_VALID)]
valid = data5[(data5['Date'] >= pd.to_datetime(MIN_VALID)) & (data5['Date'] <= pd.to_datetime(MAX_VALID))]
# Add noice
#traint['Confirmed'] = traint['Confirmed'].apply(lambda x: x + random.random())
traint.sample(3)

,Country/Region,Province/State,Date,Confirmed,Deaths,Health_GDP,Health_USD,Physicians,Nurse,Age_old/new,Smoking,tests,testpop,gatheringlimit,hospibed,healthperpop,density_n,fertility_rate,land_area,median_age,migrants,population_coun,urban_pop_rate_c,world_share,Start_date,population,population_urban,population_rural,urban_pop_rate,lat,lon,continent,area,apl-driving,apl-transit,apl-walking,grocery-and-pharmacy,parks,residential,retail-and-recreation,transit-stations,workplaces,isolation,federal_district,geoname_name,global_vec_0,global_vec_1,global_vec_2,global_vec_3,global_vec_4,...,Confirmed per day_mean_8_14,Confirmed per day_mean_15_21,Confirmed per day_max_1_7,Confirmed per day_min_1_7,Confirmed per day_median_1_7,days_from_1,days_from_10,days_from_100,lag_1_cc,lag_2_cc,lag_3_cc,lag_4_cc,lag_5_cc,lag_6_cc,lag_7_cc,lag_8_cc,lag_9_cc,lag_10_cc,lag_11_cc,lag_12_cc,lag_13_cc,perc_1_cc,diff_1_cc,diff_2_cc,diff_3_cc,diff_123_cc,diff_change_1_cc,diff_change_2_cc,diff_change_12_cc,change_1_cc,change_2_cc,change_3_cc,change_1_3_cc,change_1_7_cc,day_from_max,max_value,delta_with_max,day_from_max_1,max_value_1,delta_with_max_1,apl-driving_8,apl-transit_8,apl-walking_8,grocery-and-pharmacy_8,parks_8,residential_8,retail-and-recreation_8,transit-stations_8,workplaces_8,isolation_8
4971,Costa Rica,nan,2020-03-11,13,0,7.600000,888.900000,1.100000,0.800000,0.654273,11.9,40218.117147,10403.318753,33.482838,1.200000,252.448019,100.000000,1.800000,5.106000e+04,33.000000,4200.000000,5.094118e+06,0.800000,0.000700,83.0,1.753027e+06,1312777.8,440249.282353,0.709728,9.7489,-83.7534,Americas,51100.0,82.605326,72.823825,84.835839,NaN,NaN,NaN,NaN,NaN,NaN,2.163219,NaN,NaN,0.309397,0.064277,0.09168,0.29504,0.160987,...,0.000000,0.000000,4.0,0.0,0.0,27,22,12,9.0,9.0,5.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,0.0,4.0,4.0,2.666667,0.000000,1.000000,0.500000,1.000000,1.800000,5.000000,9.000000,inf,3,4.0,0.0,2.0,4.0,-4.0,82.605326,72.823825,84.835839,NaN,NaN,NaN,NaN,NaN,NaN,2.163219
8691,Ireland,Ireland,2020-02-15,0,0,7.400000,4758.600000,3.100000,14.300000,0.892568,24.3,1784.000000,2767.817265,100.000000,2.800000,1080.443745,72.000000,1.800000,6.889000e+04,38.000000,23604.000000,4.937786e+06,0.630000,0.000600,72.0,1.753027e+06,1312777.8,440249.282353,0.709728,NaN,NaN,NaN,NaN,136.510000,125.930000,188.570000,NaN,NaN,NaN,NaN,NaN,NaN,2.163219,NaN,NaN,0.309397,0.064277,0.09168,0.29504,0.160987,...,0.000000,0.000000,0.0,0.0,0.0,8,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,0.0,0.0,23.0,0.0,0.0,138.660000,132.250000,174.380000,NaN,NaN,NaN,NaN,NaN,NaN,2.163219
23082,United States,"Nassau County, New York, United States",2020-04-08,20140,0,8.145282,2443.245513,2.815329,7.279926,1.032643,21.8,40218.117147,10403.318753,33.482838,5.479373,889.584838,172.530689,1.846099,3.190620e+06,39.649563,86384.955442,1.396698e+08,0.701473,0.017919,79.0,1.753027e+06,1312777.8,440249.282353,0.709728,NaN,NaN,NaN,NaN,82.605326,72.823825,84.835839,NaN,NaN,NaN,NaN,NaN,NaN,2.163219,NaN,NaN,0.309397,0.064277,0.09168,0.29504,0.160987,...,895.571429,443.142857,1938.0,994.0,1218.0,56,53,46,18548.0,16610.0,15616.0,14398.0,13346.0,12024.0,10587.0,9554.0,8544.0,7344.0,6445.0,5537.0,4657.0,0.010581,1938.0,994.0,1218.0,1383.333333,1.949698,0.816092,1.382895,1.116677,1.063653,1.084595,1.288234,1.941386,1,1938.0,-346.0,5.0,1437.0,501.0,82.605326,72.823825,84.835839,NaN,NaN,NaN,NaN,NaN,NaN,2.163219


In [0]:
features = [
 #'Country/Region',
 #'Province/State',
 #'Date',
 #'Confirmed',
 #'Deaths',
 'Health_GDP',
 'Health_USD',
 'Physicians',
 'Nurse',
 'Age_old/new',
 'Smoking',
 'tests',
 'testpop',
 'gatheringlimit',
 'hospibed',
 'healthperpop',
 'density_n',
 'fertility_rate',
 'land_area',
 'median_age',
 'migrants',
 'population_coun',
 'urban_pop_rate_c',
 'world_share',
 'Start_date',
 'population',
 'population_urban',
 'population_rural',
 'urban_pop_rate',
 'lat',
 'lon',
 'continent',
 'area',
 #'apl-driving',
 #'apl-transit',
 #'apl-walking',
 #'grocery-and-pharmacy',
 #'parks',
 #'residential',
 #'retail-and-recreation',
 #'transit-stations',
 #'workplaces',
 #'isolation',
 'federal_district',
 'geoname_name',
 'global_vec_0',
 'global_vec_1',
 'global_vec_2',
 'global_vec_3',
 'global_vec_4',
 'global_vec_5',
 'global_vec_6',
 'global_vec_7',
 'global_vec_8',
 'global_vec_9',
 'global_vec_10',
 'global_vec_11',
 'global_vec_12',
 'global_vec_13',
 'global_vec_14',
 'global_vec_15',
 'global_vec_16',
 'global_vec_17',
 'global_vec_18',
 'global_vec_19',
 'global_vec_20',
 'global_vec_21',
 'global_vec_22',
 'global_vec_23',
 'global_vec_24',
 'global_vec_25',
 'global_vec_26',
 'global_vec_27',
 'region_vec_0',
 'region_vec_1',
 'region_vec_2',
 'region_vec_3',
 'region_vec_4',
 'region_vec_5',
 'region_vec_6',
 'region_vec_7',
 'region_vec_8',
 'region_vec_9',
 'region_vec_10',
 'region_vec_11',
 'region_vec_12',
 'region_vec_13',
 'region_vec_14',
 'region_vec_15',
 'region_vec_16',
 'region_vec_17',
 'region_vec_18',
 'region_vec_19',
 'region_vec_20',
 'region_vec_21',
 'region_vec_22',
 'region_vec_23',
 'region_vec_24',
 'region_vec_25',
 'region_vec_26',
 'region_vec_27',
 'day',
 #'place',
 #'Confirmed per day',
 'Confirmed per day_mean_1_1',
 'Confirmed per day_mean_1_7',
 'Confirmed per day_mean_8_14',
 'Confirmed per day_mean_15_21',
 'Confirmed per day_max_1_7',
 'Confirmed per day_min_1_7',
 'Confirmed per day_median_1_7',
 'days_from_1',
 'days_from_10',
 'days_from_100',
 'lag_1_cc',
 'lag_2_cc',
 'lag_3_cc',
 'lag_4_cc',
 'lag_5_cc',
 'lag_6_cc',
 'lag_7_cc',
 'lag_8_cc',
 'lag_9_cc',
 'lag_10_cc',
 'lag_11_cc',
 'lag_12_cc',
 'lag_13_cc',
 'perc_1_cc',
 'diff_1_cc',
 'diff_2_cc',
 'diff_3_cc',
 'diff_123_cc',
 'diff_change_1_cc',
 'diff_change_2_cc',
 'diff_change_12_cc',
 'change_1_cc',
 'change_2_cc',
 'change_3_cc',
 'change_1_3_cc',
 'change_1_7_cc',
 #'day_from_max',
 #'max_value',
 #'delta_with_max',
 'day_from_max_1',
 'max_value_1',
 'delta_with_max_1',
 'apl-driving_8',
 'apl-transit_8',
 'apl-walking_8',
 'grocery-and-pharmacy_8',
 'parks_8',
 'residential_8',
 'retail-and-recreation_8',
 'transit-stations_8',
 'workplaces_8',
 'isolation_8']

In [0]:
params_sample = {
    'objective': 'regression',
    'metric': 'mse',
    'boosting': 'gbdt',
    'verbosity': 100,
    'bagging_seed': 2020,
    'random_state': 2020,
    'num_round':1000,
    'learning_rate': 0.02,
    'min_data_in_leaf': 5,
    'max_depth': 8,
    'reg_alpha': 1,
    'reg_lambda': 5,}

In [0]:
col_target = 'Confirmed per day'
col_features = [
  'lat', 'lon',
 'days_from_10',
 'Confirmed per day_mean_1_1',
 'Confirmed per day_mean_1_7',
 'Confirmed per day_mean_8_14',
 'Confirmed per day_mean_15_21',
]
col_cat = []

In [0]:
dfc = valid.copy()[['Country/Region','Province/State','Date','Confirmed']]
dfc = dfc[(dfc['Country/Region'] == 'Russia') & (dfc['Province/State'] != 'nan') & (dfc['Province/State'].apply(lambda x: ',' not in x))]

In [0]:
# For add
def mean_absolute_logarithmic_error(df_actual, df_predicted):
    actual = df_actual.astype(np.float).values[:] + 1
    predicted = df_predicted.astype(np.float).values[:] + 1
    return sum(abs(np.log10(predicted) - np.log10(actual)))

def calc_score(pred, df):
  dfc = df.copy()[['Country/Region','Province/State','Date','Confirmed per day']]
  dfc['pred'] = pred
  dfc = dfc[(dfc['Country/Region'] == 'Russia') & (dfc['Province/State'] != 'nan') & (dfc['Province/State'].apply(lambda x: ',' not in x))]
  del dfc['Country/Region']
  dfc = dfc.fillna('nan')
  dfc['Date'] = dfc['Date'].apply(lambda x: str(x))
  all_country = list(dfc['Province/State'].unique())
  dfc.set_index(['Province/State', 'Date'], inplace=True)
  contry_loss = []
  for country in all_country:
      sc = mean_absolute_logarithmic_error(
          dfc.loc[country]['Confirmed per day'],
          dfc.loc[country]['pred']
      )
      contry_loss.append(sc)
  return np.mean(contry_loss)

In [0]:
def check_new_feature(old_features, new_feature, params, train, valid, target_col, cat_col, features_add):
  trainc = train.copy()
  validc = valid.copy()

  X_train = trainc[old_features+[new_feature]+features_add]
  y_train = np.log(trainc[target_col].values.clip(0, 1e10)+1)
  train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_col)

  X_valid = validc[old_features+[new_feature]+features_add]
  y_valid = np.log(validc[target_col].values.clip(0, 1e10)+1)
  valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_col)
  model_pri = lgb.train(params, train_data)

  y_pred = np.exp(model_pri.predict(X_valid))-1
  score1 = calc_score(y_pred, validc)

  trainc[new_feature] = np.random.permutation(trainc[new_feature].values)
  X_train = trainc[old_features+[new_feature]+features_add]
  y_train = np.log(trainc[target_col].values.clip(0, 1e10)+1)
  train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_col)

  X_valid = validc[old_features+[new_feature]+features_add]
  y_valid = np.log(validc[target_col].values.clip(0, 1e10)+1)
  valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_col)
  model_pri = lgb.train(params, train_data)

  y_true = validc[target_col].values
  y_pred = np.exp(model_pri.predict(X_valid))-1
  score2 = calc_score(y_pred, validc)

  print(new_feature, score1, score2, score2 - score1)
  return score2 - score1

In [49]:
col_features2 = col_features
for feature in features:
  try:
    r = check_new_feature(col_features2, feature, params_sample, trainv, valid, col_target, col_cat, [])
    if r < -0.02:
      col_features2.append(feature)
      print('add', feature)
  except Exception as ex:
    print(ex)
    pass

Health_GDP 1.4560451128566223 1.4359245629876756 -0.020120549868946735
add Health_GDP
Health_USD 1.45424231756815 1.4428497938676978 -0.011392523700452184
Physicians 1.4417585286208237 1.438405994319215 -0.003352534301608756
Nurse 1.4492976215553293 1.4590734497411912 0.009775828185861979
Age_old/new 1.4453624852745819 1.4460983268982386 0.0007358416236566967
Smoking 1.4464832526023834 1.446179016137418 -0.0003042364649654683
tests 1.4592105152375092 1.442820473858624 -0.016390041378885112
testpop 1.4356702118209452 1.4424397556176995 0.006769543796754318
gatheringlimit 1.450029665911452 1.4431236017317386 -0.0069060641797134
hospibed 1.4526110576791489 1.4483327619626956 -0.004278295716453329
healthperpop 1.4410751830811643 1.4284514878356882 -0.01262369524547613
density_n 1.448067900283178 1.4513373968311418 0.003269496547963824
fertility_rate 1.4268992568550753 1.4508599751314442 0.02396071827636881
land_area 1.4597011770042678 1.4634241195301223 0.003722942525854478
median_age 1.43